## This lab explains simple example to build AI Agent.

In [19]:
# @title Authentication

%pip install --upgrade --quiet google-cloud-aiplatform

#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [20]:
# @title Agent initialization.

from IPython.display import display, Markdown

import vertexai
import vertexai.generative_models as generative_models
from vertexai.preview.generative_models import grounding

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    Tool
)

MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(MODEL_NAME)

In [22]:
# @title Helper function
from vertexai.generative_models import ChatSession, GenerationConfig

def interactive_chat(chat: ChatSession, prompt: str, search:bool) -> str:

    tool = None
    if search:
      tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
        "top_k": 40
    }

    tools = [tool] if search else None

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

    responses = chat.send_message(
        prompt,
        generation_config = generation_config,
        safety_settings=safety_settings,
        tools = tools,
        stream=False
    )
    return responses.text

#------------------------------------------------------------------------------------

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  return history

#------------------------------------------------------------------------------------
def query_classification(chat_history, query):

  prompt = f"""당신은 이전 대화 내용을  참조하여 <입력질문>을 분류해주는 AI Assistant 입니다.
  1. 질문 분류를 기준은 아래 <분류기준>을 참고하여 아래 내용을 포함된 정보로 분류해주세요.
     - 분류이유
     - 분류결과
     - Endpoint

  2. 질문내의 키워드를 추출해서 정리해주세요.

  <대화내용>{chat_history}</대화내용>
  <입력질문>{query}</입력질문>

  <질문분류기준> [분류기준:endpoint]
  1. 영화추천 : http://sec_vd.com/영화추천
  2. 음악추천 : http://sec_vd.com/음악추천
  3. 기타 : http://sec_vd.com/기타

  """

  response_schema = {
      "type": "OBJECT",
      "properties": {
        "reason": {"type": "STRING"},
        "classification": {
            "type": "OBJECT",
            "properties": {
              "class": {"type": "STRING"},
              "endpoint": {"type": "STRING"},
            },
        }
      }
  }

  generation_config = GenerationConfig(
        temperature=0,
        top_p=1.0,
        top_k=10,
        candidate_count=1,
        max_output_tokens=8192,
        response_mime_type="application/json",
        response_schema=response_schema

    )

  responses = model.generate_content(
      [prompt],
      generation_config=generation_config,
      stream=False,
  )

  return responses.text

#------------------------------------------------------------------------------------

def agent_1(chat_history, query):

  prompt =f"""
          당신은 영화정보를 제공하는AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 영화 2~3개와 이유를 2줄 이내로 먼저 답변 해주세요.
          2. 영화를 선정 하는 이유를 설명 해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}
  """
  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],

  )

  return responses.text

def agent_2(chat_history, query):

  prompt =f"""
          당신은 음악정보를 제공하는 AI 어시스턴트 입니다. 답변은 아래 내용을 따라서 답해주세요.

          1. 오늘 날씨에 맞는 음악을 추천해주세요.
          2. 추천하는 이유를 자세히 설명해주세요.
          3. 전체적인 답변은 간략하게 최대 10줄을 넘지 않게 해주세요.

          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_3(chat_history, query):

  prompt =f"""
          당신은 일상적인 대화가 가능한 AI Agent 입니다.
          가벼운 대화를 진행하되 혹시 질문이 있을 경우 해당 질문에 대해서 답변해주세요.
          전체 답변은 10줄을 넘지 않게 해주세요.
          <질문> : {query}

  """

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  responses = model.generate_content(
      [prompt],
      tools=[tool],
  )

  return responses.text

def agent_orchestration(chat_history, query):

  classified = query_classification(chat_history, query)
  task = eval(classified)['classification']['class']

  print(f"Agent Task : {task}")

  if task == '영화추천':
    return agent_1(chat_history, query)
  elif task == '음악추천':
    return agent_2(chat_history, query)
  else:
    return agent_3(chat_history, query)


In [23]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  chat_history = get_chat_history(chat)

  response = agent_orchestration(chat_history, query)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 안녕하세요. 
Agent Task : 기타


AI Agent : 안녕하세요! 😊  날씨가 쌀쌀해졌네요. 오늘 하루는 어떻게 보내셨어요? 저는 오늘도 열심히 배우고 있답니다. 혹시 궁금한 점이 있다면 언제든지 물어봐 주세요. 😄 


------------------------------------ 
사용자: 화성에서 감자캐는 영화가 뭐야 ? 과학자가 있던데..
Agent Task : 영화추천


AI Agent : "마션"과 "인터스텔라"를 추천합니다. "마션"은 화성에 고립된 우주 비행사가 감자를 재배하며 생존하는 이야기이며, "인터스텔라"는 식량 부족 문제를 해결하기 위해 새로운 행성을 찾아 떠나는 우주 비행사들의 이야기입니다.

두 영화 모두 과학적 사실을 바탕으로 만들어졌으며, 우주에서의 생존과 인류의 미래에 대한 흥미로운 질문을 던져줍니다. 특히, "마션"은 화성에서 감자를 재배하는 장면을 사실적으로 묘사하며, 과학적 상상력을 자극합니다. "인터스텔라"는 거대한 블랙홀과 웜홀 등 우주의 신비를 보여주며, 우주에 대한 경외감을 느끼게 해줍니다.  


------------------------------------ 
사용자: 마션에서 주인공 이름은 ?
Agent Task : 영화


...

------------------------------------ 
사용자: 눈오는 날씨에 음악추천해줘
Agent Task : 음악추천


AI Agent : 눈 오는 날에는 따뜻하고 포근한 감성을 자극하는 음악이 좋겠죠? 

**정승환의 "눈사람"** 이라는 노래를 추천해 드릴게요. 

이 노래는 아이유가 작사한 동화적인 가사와 김제휘가 작곡한 레트로풍의 멜로디가 조화를 이루어 눈 내리는 풍경과 딱 맞는 분위기를 선사합니다. 

따뜻한 목소리로 부드럽게 펼쳐지는 멜로디는 겨울밤의 차가운 공기를 녹여주는 듯한 느낌을 주고, 눈 내리는 창밖을 바라보며 듣기에 딱 좋은 곡이에요. 

눈 내리는 날, 정승환의 "눈사람"을 들으며 따뜻한 차 한 잔과 함께 여유로운 시간을 보내시길 바랍니다. 


------------------------------------ 
사용자: recommend a music for today's weather in Suwon in South Korea
Agent Task : 음악추천


AI Agent : 수원의 오늘 날씨는 약한 비가 내리고 기온은 2.7℃라고 합니다. 쌀쌀한 날씨에 어울리는 따뜻하고 감성적인 음악을 추천해 드릴게요! 

**잔잔한 피아노 선율과 감성적인 보컬이 조화를 이루는 '유재하 - 사랑하기 때문에'**를 추천합니다. 

비 오는 날, 차분한 분위기 속에서 듣기 좋은 곡으로, 유재하 특유의 애절하면서도 아름다운 목소리가 가슴을 따뜻하게 녹여줄 거예요.  따뜻한 차 한 잔과 함께 감성적인 음악을 즐기며 오늘 하루를 마무리하는 건 어떨까요? 


------------------------------------ 
사용자: 종료
